In [44]:
# 다 대 일(many-to-one) 구조의 RNN을 사용하여 문맥을 반영해서 텍스트를 생성하는 모델
# 데이터에 대한 이해화 전처리
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [7]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1 # 패딩을 위한 0을 고려하여 +1
print('단어 집합의 크기 : %d' % vocab_size)
print(tokenizer.word_index)

단어 집합의 크기 : 12
{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [16]:
sequences = list()
for line in text.split('\n'): # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    print(encoded, len(encoded))
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1] # 전체 문장의 앞의 단어들을 전부 고려하여 학습하도록 데이터를 구성
        print('seq', sequence)
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))
print(sequences) # 맨 우축에 있는 단어에 대해서만 레이블로 분리해야함

[2, 3, 1, 4, 5] 5
seq [2, 3]
seq [2, 3, 1]
seq [2, 3, 1, 4]
seq [2, 3, 1, 4, 5]
[] 0
[6, 1, 7] 3
seq [6, 1]
seq [6, 1, 7]
[] 0
[8, 1, 9, 10, 1, 11] 6
seq [8, 1]
seq [8, 1, 9]
seq [8, 1, 9, 10]
seq [8, 1, 9, 10, 1]
seq [8, 1, 9, 10, 1, 11]
[] 0
학습에 사용할 샘플의 개수: 11
[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [22]:
# 전체 샘플에 대해 길이를 일치 시키자 가장 긴 샘플을 기준으로한다.
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre') # pre 이므로 max_len 6보다 짧으면 앞에 0으로 채움
print(sequences)

샘플의 최대 길이 : 6
[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [40]:
#각 샘플의 마지막 단어를 레이블로 분리
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

print(X)
print(y, y.shape)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11] (11,)


In [41]:
# 레이블에 대해 원-핫인코딩
y = to_categorical(y, num_classes=vocab_size)

In [43]:
print(y.shape)
print(y) # 맨 앞은 0으로 시작한다. 그러므로 (11, 12)

(11, 12)
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [50]:
# 모델 설계 및 훈련
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

# 다중 클래스 분류의 문제의 경우 출력층엔 소프트맥스 회귀 -> 활성화 함수: 소프트맥스 함수
# 손실 함수로는 크로스 엔트로피 함수 사용
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim)) # 임베딩 벡터의 차원은 10차원(왜? 몰라)
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax')) # 출력층은 단어 집합 크기만큼의 뉴런을 배치
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4789 - accuracy: 0.2727 - 541ms/epoch - 541ms/step
Epoch 2/200
1/1 - 0s - loss: 2.4642 - accuracy: 0.2727 - 3ms/epoch - 3ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4493 - accuracy: 0.2727 - 3ms/epoch - 3ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4341 - accuracy: 0.2727 - 3ms/epoch - 3ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4186 - accuracy: 0.2727 - 3ms/epoch - 3ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4026 - accuracy: 0.3636 - 3ms/epoch - 3ms/step
Epoch 7/200
1/1 - 0s - loss: 2.3860 - accuracy: 0.3636 - 3ms/epoch - 3ms/step
Epoch 8/200
1/1 - 0s - loss: 2.3688 - accuracy: 0.3636 - 3ms/epoch - 3ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3510 - accuracy: 0.3636 - 3ms/epoch - 3ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3324 - accuracy: 0.3636 - 3ms/epoch - 3ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3131 - accuracy: 0.3636 - 4ms/epoch - 4ms/step
Epoch 12/200
1/1 - 0s - loss: 2.2929 - accuracy: 0.3636 - 3ms/epoch - 3ms/step
Epoch 13/200
1/1 - 0s - loss: 2.2718 - accuracy: 0.3636 -

In [51]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 10)          120       
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                1376      
                                                                 
 dense_1 (Dense)             (None, 12)                396       
                                                                 
Total params: 1892 (7.39 KB)
Trainable params: 1892 (7.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1) # 예측 값 중에 제일 정확한 값의 인덱스 추출

        for word, index in tokenizer.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' ' + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [60]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에 있는 말이 뛰고 있다
